In [1]:
from collections import Counter
import json

In [10]:
def analyze(fname):
    with open(fname) as f:
        raw = json.load(f)

    # Find all JS function calls
    fcalls = [x for x in raw if x['name'] == 'FunctionCall']
    fcalls_by_ids = {}
    for c in fcalls:
        key = '{}-{}'.format(c['pid'], c['tid'])
        fcalls_by_ids.setdefault(key, []).append(c)

    for k, v in fcalls_by_ids.items():
        # See https://docs.google.com/document/d/1CvAClvFfyA5R-PhYUmn5OOQtYMH4h6I0nSsKchNAySU/preview
        # It mentioned how  Duration Events work.
        # But it never confirmed that 'FunctionCall' events do not nest when pid&tid do not change
        # This confirms that they do not nest
        print('Checking sanity of', k)
        assert('BB' not in ''.join(x['ph'] for x in v))
        assert('EE' not in ''.join(x['ph'] for x in v))
        if v[0]['ph'] =='B':
            assert ''.join(x['ph'] for x in v[:2*(len(v)//2)]) == 'BE' * (len(v)//2)
        if v[0]['ph'] =='E':
            assert ''.join(x['ph'] for x in v[1:2*(len(v)//2 + 1) - 1]) == 'BE' * (len(v)//2)
        print(' + this thread checks out')
        if len(v) %2 != 0:
            print(' * remaining event: ', v[-1]['ph'])
    print('OK')

    # Aggregate info about JS Function calls
    callFrames = Counter()
    callFunctionNames = Counter()
    callURLs = Counter()

    for k, v in fcalls_by_ids.items(): 
        bs = [x for i, x in enumerate(v) if i % 2 == 0]
        es = [x for i, x in enumerate(v) if i % 2 != 0]
        for b, e in zip(bs, es):
            info = b.get('args', {}).get('data', {})
            info.update(e.get('args', {}).get('data', {}))
            duration = e.get('tts') - b.get('tts')
            if not info: continue
            if type(duration) is not int: continue
            callFrames[info.get('frame')] += duration / 1e6
            callFunctionNames[info.get('functionName')] += duration / 1e6
            callURLs[info.get('url')] += duration / 1e6

    # ! callFrames may be major indicator: If 'None' frame uses too much time, it's usual.
    print(callFrames)
    print(callFunctionNames)
    print(callURLs)

In [11]:
analyze('./miner1-tracing.json')

Checking sanity of 14135-48387
 + this thread checks out
 * remaining event:  B
Checking sanity of 14135-775
 + this thread checks out
Checking sanity of 14135-84483
 + this thread checks out
 * remaining event:  B
Checking sanity of 14135-53251
 + this thread checks out
 * remaining event:  B
Checking sanity of 14135-76291
 + this thread checks out
 * remaining event:  B
Checking sanity of 14135-81923
 + this thread checks out
 * remaining event:  B
Checking sanity of 14135-51459
 + this thread checks out
 * remaining event:  B
Checking sanity of 14135-57091
 + this thread checks out
 * remaining event:  B
Checking sanity of 14135-72963
 + this thread checks out
 * remaining event:  B
OK
Counter({None: 141.75913399999988, '4A3278310ADC1F4D8DE8FE0AD4FD8528': 0.13138600000000003, 'FDE5E38EF5D4E19DD0D2040B43FC16E9': 0.002773999999999999})
Counter({'CryptonightWASMWrapper.workThrottled': 117.92003399999975, 'CryptonightWASMWrapper.onMessage': 23.839099999999988, 'MinerUI.updateStats': 0.0

In [12]:
analyze('./miner2-tracing.json')

Checking sanity of 16739-31235
 + this thread checks out
 * remaining event:  B
Checking sanity of 16739-44291
 + this thread checks out
 * remaining event:  B
Checking sanity of 16739-45571
 + this thread checks out
 * remaining event:  B
Checking sanity of 16739-47107
 + this thread checks out
 * remaining event:  B
Checking sanity of 16739-775
 + this thread checks out
OK
Counter({None: 116.07749799999998, 'E15AF5E14C07CC3EC5C565C90EC81B60': 0.09576399999999986})
Counter({'CryptonightWASMWrapper.onMessage': 116.07749799999998, 'u': 0.07565199999999994, 'JobThread.onReceiveMsg': 0.011430999999999997, 'Miner._updateTabs': 0.006464, 'Miner._onMessage': 0.0013039999999999998, 'Miner._onOpen': 0.000465, 'JobThread.onReady': 0.00044800000000000005})
Counter({'blob:https://eenteslavoorsimon.be/68ddd1ce-cb24-493f-9bdb-3e0c1faf7b19': 116.07749799999998, 'https://eenteslavoorsimon.be/js/main.44c9d0c64f7ccf8c9c0e.js': 0.07565199999999994, 'https://coinhive.com/lib/coinhive.min.js': 0.020112000

In [13]:
# Youtube can be told apart from miners by looking at callFrame[None]
analyze('./youtube1-tracing.json')

Checking sanity of 14960-86547
 + this thread checks out
Checking sanity of 14960-775
 + this thread checks out
OK
Counter({'967D17AB3B4B023C7BFEBA556D53B073': 1.5815709999999896, None: 0.004132})
Counter({'': 0.8633839999999994, 'F.H': 0.29600200000000004, 'observe.characterData': 0.10563299999999999, 'g.h.DL': 0.05702000000000002, 'SF.C': 0.033007000000000127, 'Polymer.Base.createdCallback': 0.031948000000000004, 'onYtAction_': 0.030808000000000002, 'g.h.Vx': 0.029967999999999998, 'Vh.checkPosition_': 0.02848300000000002, 'PO.update': 0.026033999999999998, 'g.h.fu': 0.021923, 'Polymer.Base.attributeChangedCallback': 0.008670999999999998, 'Zi': 0.008525, 'g.h.KM': 0.008314, 'g.onreadystatechange': 0.0057729999999999995, 'createdCallback': 0.005133999999999999, 'Polymer.Base.attachedCallback': 0.004188000000000001, 'g.h.KP': 0.003816, 'e': 0.003004, 'm': 0.002429, 'self.onmessage': 0.002316, 'MP.B': 0.001264, 'doAfterRenderRequest_': 0.0011129999999999998, 'g.h.je': 0.00093900000000000

In [14]:
# this is a weird site. Most computation goes to an unamed JS function
# But it's not running anything on callFrame[None]
# Also, urls 
analyze('./news-with-ads1-tracing.json')

Checking sanity of 15287-83483
 + this thread checks out
 * remaining event:  E
Checking sanity of 15287-775
 + this thread checks out
Checking sanity of 15287-49175
 + this thread checks out
OK
Counter({'209D8579153CE5091DB2E88AD7D93BC0': 0.27787099999999915, '21085C3BC727D8F2CE4128DB012F22B0': 0.146941, '9412A17B71DB48522F61A5AC6A58608C': 0.11231400000000001, None: 0.046113999999999995, 'CEAC92BE92E735E0A220AFB0DC22F2B3': 0.03516000000000002, '935F1CF24351B71C26AFCC7B690E30BD': 0.031556000000000015, '8C72FD770D19DD1CAD7C7C53775886F0': 0.028974000000000007, 'C949ABD8FD2BEF95355C4C3DC670A82F': 0.023388999999999993, '5DD0CC3C34BF577D83063E3EB96AFC20': 0.005594000000000001, 'C91BDD86208A564F5A6D1A98468C3B46': 0.002064, '0EC8DA54DF14AA995D410DE86D42D992': 0.001811, '96465CF845BD95C2A73F9640AF328799': 0.00128, '6DC06F77DB00C2746AC4AD42959B7702': 0.0010550000000000002, '6A4872D7D0573164DF809754B34169AE': 0.000913, '30A715A93CB179C12ADF03D39FF3FDE1': 0.000507, '0466FDDEBAE926552E94E659D1871C

limit_output extension: Maximum message size of 5000 exceeded with 7778 characters